<a href="https://colab.research.google.com/github/sarahelshabrawy/AP-Training-Project/blob/master/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install biopython

In [ ]:
pip install pyunpack

In [ ]:
pip install patool

In [ ]:
from Bio import SeqIO
import pandas as pd
import numpy as np
from keras.utils import np_utils
import tensorflow as tf
from pyunpack import Archive
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout,Reshape
import itertools
import re


In [ ]:
def prepare_data():
    # Archive('all.7z').extractall('/content')
    # Archive('Dataset/all.7z').extractall('Dataset')
    identifiers = []
    lengths = []
    genome_sequence = []
    prog = 1;
    # for record in SeqIO.parse("genomic.fna", "fasta"):
   
    for record in SeqIO.parse("egypt.fasta", "fasta"):
        identifiers.append(record.id)
        RNA = "".join(record.seq)
        RNA = clean_RNA(RNA)
        genome_sequence.append(RNA)
        lengths.append(len(RNA))
        if(prog == 100) :
          break
        prog = prog +1
        print(prog)
    
    # converting lists to pandas Series
    s1 = pd.Series(identifiers, name='ID')
    s2 = pd.Series(lengths, name='length')
    s3 = pd.Series(genome_sequence, name='seq')
    # Gathering Series into a pandas DataFrame and rename index as ID column
    Qfasta = pd.DataFrame(dict(ID=s1, length=s2, seq=s3)).set_index(['ID'])
    # max_len = Qfasta["length"].max()
    # print(max_len)
    # Qfasta.to_csv('dataframe.csv')
    return Qfasta

In [ ]:
def clean_RNA(RNA) :
  regex = r"([^ACTG])"
  result = re.sub(regex, "", RNA, 0, re.MULTILINE)
  return result

In [ ]:
def encode_data(df):
    encoded_data = []
    prog = 1
    for seq in df['seq']:
        input_tokens = [seq[i:i + 3] for i in range(0, len(seq) - len(seq) % 3, 3)]
        encoded_data.append(encode(dictionary,input_tokens))
        print(prog)
        prog = prog + 1
    encoded_data = tf.keras.preprocessing.sequence.pad_sequences(encoded_data,padding='post')
    #split dataset
    train_size = int(len(encoded_data) * 0.67)
    test_size = len(encoded_data) - train_size
    train, test = encoded_data[0:train_size,:], encoded_data[train_size:len(encoded_data),:]
    look_back = 10
    trainX, trainY = create_dataset(train, look_back)
    testX, testY = create_dataset(test, look_back)

    new_dimen = trainX.shape[2]*trainX.shape[3]
    trainX = np.reshape(trainX, (trainX.shape[0],look_back,new_dimen))
    trainY = np.reshape(trainY,(trainX.shape[0],new_dimen))
    testX = np.reshape(testX, (testX.shape[0], look_back,new_dimen))
    testY = np.reshape(testY,(testX.shape[0],new_dimen))

    return trainX, trainY,testX, testY

def create_dataset(dataset, look_back = 10):
    dataX, dataY = [],[]
    for i in range(len(dataset)-look_back-1):
      a= dataset[i:(i+look_back)]
      dataX.append(a)
      dataY.append(dataset[i+look_back])
      # print(dataset[i+look_back].shape)
    return np.array(dataX), np.array(dataY)
 
def encode(dictionary, tokens):
    data = []
    for key in tokens:
        data.append(dictionary[key])
    encoded = np_utils.to_categorical(data)
    # encoded = data
    return encoded

# trainX, trainY,testX, testY = encode_data(dataset)
# print(trainY.shape)
# print(trainX.shape)

In [ ]:
def create_dictionary(n, seq):
    perms = []
    for p in itertools.product(seq, repeat=n):
        perms.append("".join(p))
    dictionary = {}
    for (token, i) in zip(perms, range(1, len(perms)+1)):
        dictionary[token] = i
    return perms, dictionary

perms, dictionary = create_dictionary(3,"ACTG")
print(dictionary)

{'AAA': 1, 'AAC': 2, 'AAT': 3, 'AAG': 4, 'ACA': 5, 'ACC': 6, 'ACT': 7, 'ACG': 8, 'ATA': 9, 'ATC': 10, 'ATT': 11, 'ATG': 12, 'AGA': 13, 'AGC': 14, 'AGT': 15, 'AGG': 16, 'CAA': 17, 'CAC': 18, 'CAT': 19, 'CAG': 20, 'CCA': 21, 'CCC': 22, 'CCT': 23, 'CCG': 24, 'CTA': 25, 'CTC': 26, 'CTT': 27, 'CTG': 28, 'CGA': 29, 'CGC': 30, 'CGT': 31, 'CGG': 32, 'TAA': 33, 'TAC': 34, 'TAT': 35, 'TAG': 36, 'TCA': 37, 'TCC': 38, 'TCT': 39, 'TCG': 40, 'TTA': 41, 'TTC': 42, 'TTT': 43, 'TTG': 44, 'TGA': 45, 'TGC': 46, 'TGT': 47, 'TGG': 48, 'GAA': 49, 'GAC': 50, 'GAT': 51, 'GAG': 52, 'GCA': 53, 'GCC': 54, 'GCT': 55, 'GCG': 56, 'GTA': 57, 'GTC': 58, 'GTT': 59, 'GTG': 60, 'GGA': 61, 'GGC': 62, 'GGT': 63, 'GGG': 64}


In [ ]:
dataset = prepare_data()
trainX, trainY,testX, testY = encode_data(dataset)

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [ ]:
print(trainX.shape)
print(testX.shape)

print(trainY.shape)
model = Sequential()
model.add(LSTM(units = 50, activation = 'softmax', return_sequences = True,input_shape=(trainX.shape[1], trainX.shape[2])))
model.add(Dropout(0.2))

# model.add(LSTM(units = 60, activation = 'softmax', return_sequences = True))
# model.add(Dropout(0.3))

# model.add(LSTM(units = 80, activation = 'softmax', return_sequences = True))
# model.add(Dropout(0.4))

model.add(LSTM(units = 120, activation = 'softmax'))
model.add(Dropout(0.5))
model.add(Dense(units = trainX.shape[2]))
# model.add(Reshape((10, 65)))
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics = ['categorical_accuracy'])
model.summary()
model.output_shape
model.fit(trainX, trainY, epochs=20, batch_size=20, verbose=2)
# model.save('/content')

(56, 10, 647855)
(22, 10, 647855)
(56, 647855)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 10, 50)            129581200 
_________________________________________________________________
dropout (Dropout)            (None, 10, 50)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 120)               82080     
_________________________________________________________________
dropout_1 (Dropout)          (None, 120)               0         
_________________________________________________________________
dense (Dense)                (None, 647855)            78390455  
Total params: 208,053,735
Trainable params: 208,053,735
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
3/3 - 20s - loss: 127281.9453
Epoch 2/20
3/3 - 17s

# Model Evaluation 

In [ ]:
accuracy = model.evaluate(testX,testY,verbose = 2)
print(accuracy)

1/1 - 2s - loss: 132363.6250
132363.625


In [ ]:
def decode(input) :
  output = []
  print(len(input))
  for i in  range(0, len(input)):
          if input[i] != 0 : 
            output.append(perms[input[i]-1])
  return "".join(output)

In [ ]:
prediction = model.predict(testX)
print(prediction.shape)
sequence_length = int(trainX.shape[2]/65)
prediction = np.reshape(prediction,(testX.shape[0],sequence_length,65))
testY = np.reshape(testY,(testX.shape[0],sequence_length,65))
print(decode(np.argmax(prediction[0], axis=1)))
print(decode(np.argmax(testY[0], axis=1)))

(22, 647855)
9967
...
9967
...


In [ ]:
# Archive('dataframe.7z').extractall('/content')
# dataset = pd.read_csv('dataframe.csv')

# print(trainY.shape)
# print(trainX.shape)

In [ ]:
print(testY.shape)

(22, 9967, 65)
